In [1]:
import pandas as pd
import numpy as np
import sys
import random
import os
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-07-18 00:14:33.799753: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 00:14:33.860919: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-18 00:14:33.863129: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-18 00:14:33.863140: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

#### The data

In [3]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [4]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [5]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


#### Model training and validation

In [6]:
def lstm(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
#     model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.LSTM(50),
#     tf.keras.layers.Dense(1)
# ])
    
    input_layer = Input(shape=(X_train.shape[1],))
    reshaped_input = tf.expand_dims(input_layer, axis=1)
    
    lstm_layer1 = LSTM(units=50, return_sequences=True)(reshaped_input)
    dropout_layer1 = Dropout(dropout)(lstm_layer1)

    lstm_layer2 = LSTM(units=50)(dropout_layer1)
    dropout_layer2 = Dropout(dropout)(lstm_layer2)
    
    output_layer = Dense(units=1)(dropout_layer2)

    model = Model(inputs=input_layer, outputs=output_layer) 

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_split=0.2)
    
#     model = keras.Sequential()
#     model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
#     model.add(LSTM(50, return_sequences=True, input_shape=(window_size, x_train.shape[-1])))
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM((window_size * 2), return_sequences=True))) 
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM(window_size, return_sequences=False))) 
#     model.add(Dense(units=1))
    return model

In [7]:
# def scale_data(X):
#     scaler = MinMaxScaler()
#     X_scaled = X.copy()
#     X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
#     return X_scaled

In [8]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
    assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
    assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
    assert(len(train_df.longitude.unique()) == len(longitudes)-1)
    assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
#     X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
#     X_test_scaled = scale_data(X_test)
    
    model = lstm(X_train, y_train, dropout=0.1)
    y_pred = model.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [9]:
rmse_list, mape_list = [], []
for i in range(len(latitudes)):
    rmse, mape = cross_validation(final_df, i)
    rmse_list.append(rmse)
    mape_list.append(mape)
    print(f'{device_ids[i]} successful')

2023-07-18 00:14:40.365604: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-18 00:14:40.365636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PL1207-PRO.paris.inria.fr
2023-07-18 00:14:40.365641: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PL1207-PRO.paris.inria.fr
2023-07-18 00:14:40.365751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.125.6
2023-07-18 00:14:40.365776: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.125.06  Release Build  (dvs-builder@U16-A23-14-1)  Tue May 30 05:12:48 UTC 2023
GCC version:  gcc ve

Epoch 1/1000
289/289 [==============================] - 2s 3ms/step - loss: 694.9516 - val_loss: 852.1271
Epoch 2/1000
289/289 [==============================] - 1s 2ms/step - loss: 431.2969 - val_loss: 681.5632
Epoch 3/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.8543 - val_loss: 672.2404
Epoch 4/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.2550 - val_loss: 669.6503
Epoch 5/1000
289/289 [==============================] - 1s 2ms/step - loss: 413.5133 - val_loss: 673.4048
Epoch 6/1000
289/289 [==============================] - 1s 2ms/step - loss: 412.2217 - val_loss: 672.6486
Epoch 7/1000
289/289 [==============================] - 1s 2ms/step - loss: 412.7553 - val_loss: 668.3574
Epoch 8/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.4167 - val_loss: 671.5829
Epoch 9/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.2859 - val_loss: 671.6776
Epoch 10/1000
289/289 [=======================

289/289 [==============================] - 1s 3ms/step - loss: 413.0648 - val_loss: 671.6243
Epoch 78/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.9593 - val_loss: 671.6257
Epoch 79/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.8364 - val_loss: 671.2745
Epoch 80/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.4764 - val_loss: 674.2246
Epoch 81/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.0963 - val_loss: 670.6092
Epoch 82/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.7950 - val_loss: 668.2852
Epoch 83/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.7416 - val_loss: 671.0858
Epoch 84/1000
289/289 [==============================] - 1s 2ms/step - loss: 412.9218 - val_loss: 671.6980
Epoch 85/1000
289/289 [==============================] - 1s 2ms/step - loss: 412.5948 - val_loss: 669.9186
Epoch 86/1000
289/289 [============================

289/289 [==============================] - 1s 3ms/step - loss: 413.3743 - val_loss: 670.8391
Epoch 154/1000
289/289 [==============================] - 1s 3ms/step - loss: 413.6115 - val_loss: 671.8170
Epoch 155/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.8165 - val_loss: 671.7192
Epoch 156/1000
289/289 [==============================] - 1s 2ms/step - loss: 413.3006 - val_loss: 671.7897
Epoch 157/1000
289/289 [==============================] - 1s 2ms/step - loss: 412.6409 - val_loss: 673.4435
Epoch 158/1000
289/289 [==============================] - 1s 2ms/step - loss: 412.2475 - val_loss: 672.9436
Epoch 159/1000
289/289 [==============================] - 1s 3ms/step - loss: 411.9346 - val_loss: 672.7390
Epoch 160/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.0521 - val_loss: 672.3123
Epoch 161/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.7286 - val_loss: 670.5284
Epoch 162/1000
289/289 [===================

Epoch 229/1000
289/289 [==============================] - 1s 2ms/step - loss: 412.1260 - val_loss: 671.2851
Epoch 230/1000
289/289 [==============================] - 1s 2ms/step - loss: 412.7100 - val_loss: 668.9734
Epoch 231/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.8650 - val_loss: 671.7304
Epoch 232/1000
289/289 [==============================] - 1s 2ms/step - loss: 412.2124 - val_loss: 666.6350
Epoch 233/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.6076 - val_loss: 671.6459
Epoch 234/1000
289/289 [==============================] - 1s 2ms/step - loss: 413.5485 - val_loss: 671.6458
Epoch 235/1000
289/289 [==============================] - 1s 2ms/step - loss: 412.7284 - val_loss: 673.3334
Epoch 236/1000
289/289 [==============================] - 1s 3ms/step - loss: 412.6068 - val_loss: 670.6839
Epoch 237/1000
289/289 [==============================] - 1s 2ms/step - loss: 412.5388 - val_loss: 672.8024
Epoch 238/1000
289/289 [====

...

Epoch 647/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.3348 - val_loss: 393.6104
Epoch 648/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.4628 - val_loss: 393.6503
Epoch 649/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.0414 - val_loss: 393.9232
Epoch 650/1000
315/315 [==============================] - 1s 2ms/step - loss: 379.0966 - val_loss: 393.8816
Epoch 651/1000
315/315 [==============================] - 1s 3ms/step - loss: 378.3152 - val_loss: 393.8227
Epoch 652/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.6512 - val_loss: 394.0461
Epoch 653/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.6684 - val_loss: 393.7320
Epoch 654/1000
315/315 [==============================] - 1s 2ms/step - loss: 379.2493 - val_loss: 394.8185
Epoch 655/1000
315/315 [==============================] - 1s 3ms/step - loss: 378.1992 - val_loss: 393.7389
Epoch 656/1000
315/315 [====

Epoch 723/1000
315/315 [==============================] - 1s 2ms/step - loss: 378.8323 - val_loss: 393.4636
Epoch 724/1000
315/315 [==============================] - 1s 2ms/step - loss: 379.3210 - val_loss: 393.4684
Epoch 725/1000
315/315 [==============================] - 1s 2ms/step - loss: 379.9503 - val_loss: 393.8971
Epoch 726/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.0865 - val_loss: 394.2093
Epoch 727/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.5405 - val_loss: 393.8776
Epoch 728/1000
315/315 [==============================] - 1s 3ms/step - loss: 378.8544 - val_loss: 393.4457
Epoch 729/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.2380 - val_loss: 393.4590
Epoch 730/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.3336 - val_loss: 393.4182
Epoch 731/1000
315/315 [==============================] - 1s 3ms/step - loss: 378.7755 - val_loss: 393.5938
Epoch 732/1000
315/315 [====

Epoch 799/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.5438 - val_loss: 393.8919
Epoch 800/1000
315/315 [==============================] - 1s 3ms/step - loss: 378.6690 - val_loss: 393.6782
Epoch 801/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.6743 - val_loss: 393.4256
Epoch 802/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.2833 - val_loss: 393.4562
Epoch 803/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.2601 - val_loss: 393.4281
Epoch 804/1000
315/315 [==============================] - 1s 3ms/step - loss: 378.7842 - val_loss: 393.4991
Epoch 805/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.2773 - val_loss: 393.4240
Epoch 806/1000
315/315 [==============================] - 1s 3ms/step - loss: 378.7522 - val_loss: 393.6211
Epoch 807/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.8765 - val_loss: 393.4586
Epoch 808/1000
315/315 [====

Epoch 875/1000
315/315 [==============================] - 1s 2ms/step - loss: 379.1137 - val_loss: 393.7620
Epoch 876/1000
315/315 [==============================] - 1s 3ms/step - loss: 378.7641 - val_loss: 393.4318
Epoch 877/1000
315/315 [==============================] - 1s 2ms/step - loss: 379.3359 - val_loss: 393.6542
Epoch 878/1000
315/315 [==============================] - 1s 2ms/step - loss: 379.2291 - val_loss: 394.0508
Epoch 879/1000
315/315 [==============================] - 1s 2ms/step - loss: 379.1112 - val_loss: 393.6589
Epoch 880/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.2939 - val_loss: 394.3704
Epoch 881/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.0300 - val_loss: 393.4175
Epoch 882/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.2218 - val_loss: 393.5898
Epoch 883/1000
315/315 [==============================] - 1s 3ms/step - loss: 378.8314 - val_loss: 393.4951
Epoch 884/1000
315/315 [====

Epoch 951/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.6329 - val_loss: 393.4485
Epoch 952/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.2444 - val_loss: 393.5327
Epoch 953/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.0429 - val_loss: 393.4512
Epoch 954/1000
315/315 [==============================] - 1s 2ms/step - loss: 379.4863 - val_loss: 393.5695
Epoch 955/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.4615 - val_loss: 393.4172
Epoch 956/1000
315/315 [==============================] - 1s 2ms/step - loss: 378.8676 - val_loss: 394.0465
Epoch 957/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.3942 - val_loss: 393.9090
Epoch 958/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.2548 - val_loss: 393.4709
Epoch 959/1000
315/315 [==============================] - 1s 3ms/step - loss: 378.7789 - val_loss: 393.9129
Epoch 960/1000
315/315 [====

In [10]:
mean_rmse, mean_mape = np.mean(rmse_list), np.mean(mape_list)          
mean_rmse, mean_mape

(20.143940835862075, 0.5685899071696793)

In [11]:
rmse_list

[11.166335646876496,
 27.958452514345076,
 15.948686762025782,
 21.45535777257762,
 12.668033435101844,
 25.628339386102763,
 20.239053269447822,
 13.043423726071753,
 25.93960731449374,
 27.39211853157783]

In [12]:
mape_list

[0.6970206005468087,
 0.4580998187346191,
 0.6611245424208663,
 0.4506518047767067,
 0.5659288216841764,
 0.5396503755400477,
 0.5360144420389344,
 0.7348552555890581,
 0.538894080333949,
 0.5036593300316257]